In [8]:
#!pip install emmv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.dates as mdates
%matplotlib inline
warnings.filterwarnings("ignore")
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import matplotlib.pyplot as plt
from matplotlib import pyplot
#import plotly.graph_objs as go
#init_notebook_mode(connected=True)
import eif as iso
from functions import load_dataset,calc_percent_NAs, adfuller_test,data_load
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from emmv import emmv_scores

In [31]:
data = load_dataset("anomaly 3.csv")
data

name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4
Datetime,,,,,,,,
2021-03-11 10:10:00,47020.962524,2.083591,2.207742,1.494308,1.014526,NaN,23.092529,22.924228
2021-03-11 10:10:01,46970.432281,NaN,2.207284,NaN,1.014297,23.006652,23.098698,22.922855
2021-03-11 10:10:02,46931.465149,2.083860,2.207056,1.493851,1.014069,23.012825,23.106237,NaN
2021-03-11 10:10:03,46941.994858,2.084130,2.207513,1.494308,1.014526,23.013511,23.102810,22.915990
2021-03-11 10:10:04,47028.034973,NaN,2.206827,NaN,1.014297,NaN,23.090473,22.912557
...,...,...,...,...,...,...,...,...
2021-03-11 14:59:55,46302.044678,1.978631,NaN,NaN,1.011100,27.887662,27.969157,NaN
2021-03-11 14:59:56,46220.790482,1.977286,2.129547,1.477621,1.010415,27.889034,NaN,27.785660
2021-03-11 14:59:57,46267.901230,1.977016,2.129318,1.477164,NaN,27.888348,27.969842,27.793899


In [32]:
missing = calc_percent_NAs(data)
cols = data.columns
index = pd.to_datetime(data.index,utc = True)
##Missing values imputation
lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman',random_state=0)
data = imp.fit_transform(data)
data = pd.DataFrame(data, columns=cols, index = index)
data

[IterativeImputer] Completing matrix with shape (17198, 8)
[IterativeImputer] Ending imputation round 1/10, elapsed time 0.06
[IterativeImputer] Change: 185.21818094469492, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 2/10, elapsed time 0.11
[IterativeImputer] Change: 613.121087315142, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 3/10, elapsed time 0.16
[IterativeImputer] Change: 367.57762090566433, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 4/10, elapsed time 0.25
[IterativeImputer] Change: 240.76327517534, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 5/10, elapsed time 0.30
[IterativeImputer] Change: 127.88478513270789, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 6/10, elapsed time 0.34
[IterativeImputer] Change: 58.97193661217366, scaled tolerance: 47.176522064209 
[IterativeImputer] Ending imputation round 7/10, elap

name,Flow,P_1,P_3,P_4,P_5,T_2,T_3,T_4
Datetime,,,,,,,,
2021-03-11 10:10:00+00:00,47020.962524,2.083591,2.207742,1.494308,1.014526,23.026113,23.092529,22.924228
2021-03-11 10:10:01+00:00,46970.432281,2.083841,2.207284,1.496476,1.014297,23.006652,23.098698,22.922855
2021-03-11 10:10:02+00:00,46931.465149,2.083860,2.207056,1.493851,1.014069,23.012825,23.106237,22.927701
2021-03-11 10:10:03+00:00,46941.994858,2.084130,2.207513,1.494308,1.014526,23.013511,23.102810,22.915990
2021-03-11 10:10:04+00:00,47028.034973,2.083255,2.206827,1.495331,1.014297,23.010091,23.090473,22.912557
...,...,...,...,...,...,...,...,...
2021-03-11 14:59:55+00:00,46302.044678,1.978631,2.131307,1.479657,1.011100,27.887662,27.969157,27.806044
2021-03-11 14:59:56+00:00,46220.790482,1.977286,2.129547,1.477621,1.010415,27.889034,27.948747,27.785660
2021-03-11 14:59:57+00:00,46267.901230,1.977016,2.129318,1.477164,1.012492,27.888348,27.969842,27.793899


In [33]:
data = data.diff().dropna()

In [12]:
#X_train, X_test = train_test_split(data, test_size=0.4, shuffle = False,random_state = 42)

Isolation Forest

In [13]:
# Calculate Silhouette Score. Not very successful.
"""def get_score(n_estimators,max_samples,contamination, X):
    from sklearn.metrics import silhouette_score
    F2 = IsolationForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination, \
                        max_features=1.0, bootstrap=False, n_jobs=-1, verbose=0, random_state = 42)
    F2.fit(X)
    pred=F2.predict(X)
    score = silhouette_score(X, pred)
    print("n_estimators = ", n_estimators, ", max_samples=", max_samples,", contamination= ",contamination, ", score= ", score)"""

'def get_score(n_estimators,max_samples,contamination, X):\n    from sklearn.metrics import silhouette_score\n    F2 = IsolationForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination,                         max_features=1.0, bootstrap=False, n_jobs=-1, verbose=0, random_state = 42)\n    F2.fit(X)\n    pred=F2.predict(X)\n    score = silhouette_score(X, pred)\n    print("n_estimators = ", n_estimators, ", max_samples=", max_samples,", contamination= ",contamination, ", score= ", score)'

In [14]:
'''for n_estimators in [20, 50, 100,300]:
    for max_samples in [128, 256, 512]:
        for contamination in [0.05, 'auto']:
            start = time.time()
            #get_score(n_estimators, max_samples,contamination, X=X_train)
            F2 = IsolationForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination, \
                        max_features=1.0, bootstrap=False, n_jobs=-1, verbose=0, random_state = 42).fit(data)
            em, mv = emmv_scores(F2, data).values()
            end = time.time()
            print("n_estimators: ", n_estimators, ", max_samples: ", max_samples, ", contamination: ", contamination)
            print("em score: ", em, ", mv score: ", mv)
            print("Runtime: ", end-start)
            print()'''

'for n_estimators in [20, 50, 100,300]:\n    for max_samples in [128, 256, 512]:\n        for contamination in [0.05, \'auto\']:\n            start = time.time()\n            #get_score(n_estimators, max_samples,contamination, X=X_train)\n            F2 = IsolationForest(n_estimators=n_estimators, max_samples=max_samples, contamination=contamination,                         max_features=1.0, bootstrap=False, n_jobs=-1, verbose=0, random_state = 42).fit(data)\n            em, mv = emmv_scores(F2, data).values()\n            end = time.time()\n            print("n_estimators: ", n_estimators, ", max_samples: ", max_samples, ", contamination: ", contamination)\n            print("em score: ", em, ", mv score: ", mv)\n            print("Runtime: ", end-start)\n            print()'

In [34]:
F2 = IsolationForest(n_estimators=100, max_samples=512, contamination=0.02, \
max_features=1.0, bootstrap=False, n_jobs=-1, verbose=0, random_state = 42)
F2.fit(data)
pred = F2.decision_function(data)
pred= F2.predict(data)
em, mv = emmv_scores(F2, data).values()
print("em score: ", em, ", mv score: ", mv)

em score:  0.9873332395396406 , mv score:  0.01007192742996848


In [41]:
data['anomaly'] = pred

In [36]:
y_test = np.array(pd.read_csv("y_test_pred"))[:,1]


In [37]:
for i in range(len(y_test)):
    if pred[i] == -1:
        print(y_test[i], pred[i])

1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
-1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
-1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
1 -1
1 -1
1 -1
1 -1
-1 -1
-1 -1
1 -1

In [38]:
confusion_matrix(y_test, pred)
roc_auc_score(y_test, pred)



0.9918860545210437

In [39]:
confusion_matrix(y_test, pred)

array([[   66,     0],
       [  278, 16853]], dtype=int64)

In [44]:
print(data[data['anomaly']==-1])

anomaly = data[data['anomaly']==-1]
anomaly.to_csv("anomaly_iso")

name                             Flow       P_1       P_3       P_4       P_5  \
Datetime                                                                        
2021-03-11 10:11:35+00:00   33.302307 -0.026626 -0.016772  0.000457  0.000228   
2021-03-11 10:11:36+00:00  -75.946426  0.027433  0.017229  0.000229 -0.000457   
2021-03-11 10:12:45+00:00  151.604462  0.008745  0.006432 -0.000457 -0.001296   
2021-03-11 10:14:24+00:00   35.671234 -0.000269 -0.000457  0.000914  0.000685   
2021-03-11 10:14:26+00:00   51.948166  0.000538  0.000457 -0.000686 -0.000228   
...                               ...       ...       ...       ...       ...   
2021-03-11 14:56:26+00:00 -236.686796 -0.056572 -0.041996 -0.015293  0.000088   
2021-03-11 14:57:13+00:00   36.584473  0.017562  0.016588  0.023439  0.003007   
2021-03-11 14:57:14+00:00  -19.727325 -0.016755 -0.015902 -0.024536 -0.002832   
2021-03-11 14:58:43+00:00   35.746765  0.055785  0.041857  0.018277 -0.000029   
2021-03-11 14:58:44+00:00 -1